# Variational Autoencoders
## Introduction

The variational autoencoder (VAE) is arguably the simplest setup that realizes deep probabilistic modeling. Note that we’re being careful in our choice of language here. The VAE isn’t a model as such—rather the VAE is a particular setup for doing variational inference for a certain class of models. The class of models is quite broad: basically any (unsupervised) density estimator with latent random variables.

<img src="vae_model.png"  width="180" height="200">

Here we’ve depicted the structure of the kind of model we’re interested in as a graphical model. We have $N$ observed datapoints ${xi}$ Each datapoint is generated by a (local) latent random variable $Zi$. There is also a parameter $\theta$, which is global in the sense that all the datapoints depend on it (which is why it’s drawn outside the rectangle). Note that since $\theta$ is a parameter, it’s not something we’re being Bayesian about. Finally, what’s of particular importance here is that we allow for each $xi$ to depend on $zi$ in a complex, non-linear way. In practice this dependency will be parameterized by a (deep) neural network with parameters $\theta$. It’s this non-linearity that makes inference for this class of models particularly challenging.

Of course this non-linear structure is also one reason why this class of models offers a very flexible approach to modeling complex data. Indeed it’s worth emphasizing that each of the components of the model can be ‘reconfigured’ in a variety of different ways. For example:

- the neural network in $p\theta(X|z)$ can be varied in all the usual ways (number of layers, type of non-linearities, number of hidden units, etc.)

- we can choose observation likelihoods that suit the dataset at hand: gaussian, bernoulli, categorical, etc.

- we can choose the number of dimensions in the latent space

The graphical model representation is a useful way to think about the structure of the model, but it can also be fruitful to look at an explicit factorization of the joint probability density:

<img src="prob-density.png"  width="300" height="100">

The fact that $p(X|z)$ breaks up into a product of terms like this makes it clear what we mean when we call $zi$ a local random variable. For any particular $i$, only the single datapoint $xi$ depends on $zi$. As such the ${zi}$ describe local structure, i.e. structure that is private to each data point. This factorized structure also means that we can do subsampling during the course of learning. As such this sort of model is amenable to the large data setting.

That’s all there is to the model. Since the observations depend on the latent random variables in a complicated, non-linear way, we expect the posterior over the latents to have a complex structure. Consequently in order to do inference in this model we need to specify a flexibly family of guides (i.e. variational distributions). Since we want to be able to scale to large datasets, our guide is going to make use of amortization to keep the number of variational parameters under control.

Recall that the job of the guide is to ‘guess’ good values for the latent random variables—good in the sense that they’re true to the model prior and true to the data. If we weren’t making use of amortization, we would introduce variational parameters ${\lambda i}$ for each datapoint $xi$. These variational parameters would represent our belief about ‘good’ values of $zi$; for example, they could encode the mean and variance of a gaussian distribution in $zi$ space. Amortization means that, rather than introducing variational parameters, we instead learn a function that maps each $xi$ to an appropriate $\lambda i$. Since we need this function to be flexible, we parameterize it as a neural network. We thus end up with a parameterized family of distributions over the latent $z$ space that can be instantiated for all $N$ datapoint $xi$.

<img src="vae_guide.png"  width="180" height="200">

## VAE in Pyro

Let’s see how we implement a VAE in Pyro. The dataset we’re going to model is MNIST, a collection of images of handwritten digits. Since this is a popular benchmark dataset, we can make use of PyTorch’s convenient data loader functionalities to reduce the amount of boilerplate code we need to write:

In [4]:
import os

import numpy as np
import torch
from pyro.contrib.examples.util import MNIST
import torch.nn as nn
import torchvision.transforms as transforms

import pyro
import pyro.distributions as dist
import pyro.contrib.examples.util  # patches torchvision
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam

In [5]:
assert pyro.__version__.startswith('1.8.4')
pyro.distributions.enable_validation(False)
pyro.set_rng_seed(0)

smoke__test = 'CI' in os.environ

In [6]:
# for loading and batching MNIST dataset

def setup_data_louders(batch_size=128, use_cuda=False):
    root = './data'
    download = True
    trans = transforms.ToTensor()
    train_set = MNIST(root=root, train=True, transform=trans,
                      download=download)
    test_set = MNIST(root=root, train=False, transform=trans)

    kwargs = {'num_workers': 1, 'pin_memory': use_cuda}
    train_loader = torch.utils.data.Dataloader(dataset=train_set,
                                               batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.Dataloader(dataset=test_set,
                                              batch_size=batch_size, shuffle=False, **kwargs)
    return train_loader, test_loader

The main thing to draw attention to here is that we use transforms.ToTensor() to normalize the pixel intensities to the range [0.0, 1.0].

Next we define a PyTorch module that encapsulates our decoder network:

In [7]:
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim):
        super().__init__()
        #setup the two linear transformations used
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, 784)
        # setup the non-linearities
        self.softplus = nn.Softplus()
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        # define the forward computation on the latent z
        # first compute the hidden units
        hidden = self.softplus(self.fc1(z))
        # return the parameter for the output Bernoulli
        # each is of size batch_size x 784
        loc_img = self.sigmoid(self.fc21(hidden))
        return loc_img